# Installation
Just pip install:

```pip install omegaconf```

If you want to try this notebook after checking out the repository be sure to run 
```python setup.py develop``` at the repository root before running this code.

# Creating OmegaConf objects
### Empty

In [ ]:
from omegaconf import OmegaConf
conf = OmegaConf.create()
print(conf)

### From a dictionary

In [ ]:
conf = OmegaConf.create(dict(k='v',list=[1,dict(a='1',b='2')]))
print(conf.pretty())

### From a list

In [ ]:
conf = OmegaConf.create([1, dict(a=10, b=dict(a=10))])
print(conf.pretty())

### From a yaml file

In [ ]:
conf = OmegaConf.load('../source/example.yaml')
print(conf.pretty())

### From a yaml string

In [ ]:
yaml = """
a: b
b: c
list:
- item1
- item2
"""
conf = OmegaConf.create(yaml)
print(conf.pretty())

### From a dot-list

In [ ]:
dot_list = ["a.aa.aaa=1", "a.aa.bbb=2", "a.bb.aaa=3", "a.bb.bbb=4"]
conf = OmegaConf.from_dotlist(dot_list)
print(conf.pretty())

### From command line arguments

To parse the content of sys.arg:

In [ ]:
# Simulating command line arguments
import sys
sys.argv = ['your-program.py', 'server.port=82', 'log.file=log2.txt']
conf = OmegaConf.from_cli()
print(conf.pretty())

# Access and manipulation
Input yaml file:

In [ ]:
%load '../source/example.yaml'


#### Object style access:

In [ ]:
conf = OmegaConf.load('../source/example.yaml')
conf.server.port

#### dictionary style access

In [ ]:
conf['log']['rotation']

#### items in list

In [ ]:
conf.users[0]

#### Changing existing keys

In [ ]:
conf.server.port = 81

#### Adding new keys

In [ ]:
conf.server.hostname = "localhost"

#### Adding a new dictionary

In [ ]:
conf.database = {'hostname': 'database01', 'port': 3306}

#### providing default values

In [ ]:
conf.missing_key or 'a default value'
conf.get('missing_key', 'a default value')

#### Accessing mandatory values
Accessing fields with the value *???* will cause a MissingMandatoryValue exception.
Use this to indicate that the value must be set before accessing.

In [ ]:
conf.log.file

# Variable interpolation

OmegaConf support variable interpolation, Interpolations are evaluated lazily on access.

### Config node interpolation

In [ ]:
%load '../source/config_interpolation.yaml'

Interpolations are evaluated lazily on field access.<br/>
Note below that when printed the interpolations are not resolved.<br/>
They get resolved once you access them.

In [ ]:
conf = OmegaConf.load('../source/config_interpolation.yaml')
print(conf.pretty())

Primitive interpolations type is inherited from the referenced value.<br/>
Composite interpolations type is always string.


In [ ]:
# Primitive interpolation types are inherited from the referenced value
print("conf.client.server_port: ", conf.client.server_port, type(conf.client.server_port).__name__)
# Composite interpolation types are always string
print("conf.client.url: ", conf.client.url, type(conf.client.url).__name__)

### Environment variable interpolation

Environment variable interpolation is also supported.

Input yaml file:

In [ ]:
%load '../source/env_interpolation.yaml'

In [ ]:
conf = OmegaConf.load('../source/env_interpolation.yaml')
print(conf.user.name)
print(conf.user.home)

## Custom interpolations

You can add additional interpolation types using custom resolvers. This example creates a resolver that adds 10 the the given value.

In [ ]:
OmegaConf.register_resolver("plus_10", lambda x: int(x) + 10)
conf = OmegaConf.create({'key': '${plus_10:990}'})
print(conf.key)

# Merging configurations
Merging configurations enables the creation of reusable configuration files for each logical component instead of a single config file for each variation of your task.

Machine learning experiment example:
```python
conf = OmegaConf.merge(base_cfg, model_cfg, optimizer_cfg, dataset_cfg)
```

Web server configuration example:

```python
conf = OmegaConf.merge(server_cfg, plugin1_cfg, site1_cfg, site2_cfg)
```

The following example creates two configs from files, and one from the cli. It then combines them into a single object. Note how the port changes to 82, and how the users lists are combined.

In [ ]:
%load '../source/example2.yaml'

In [ ]:
%load '../source/example3.yaml'

In [ ]:
from omegaconf import OmegaConf
import sys
base_conf = OmegaConf.load('../source/example2.yaml')
second_conf = OmegaConf.load('../source/example3.yaml')

# Merge configs:
conf = OmegaConf.merge(base_conf, second_conf)

# Simulate command line arguments
sys.argv = ['program.py', 'server.port=82']
# Merge with cli arguments
conf.merge_with_cli()
print(conf.pretty())